In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
from os.path import abspath
import re
import os
import matplotlib.image
from functools import partial
from IPython.display import Image
from sklearn import linear_model
from statistics import median 
# import plotly.offline as py
import seaborn as sns
# import plotly.figure_factory as ff
# import pingouin as pg
import itertools
from tqdm import tqdm 
import time 


In [ ]:
trip17 = pd.read_csv('trippub.csv')
trip17.head()


In [ ]:
# Drop all zero values of trip dataset
trip17 = trip17.loc[trip17.WTTRDFIN > 0]

# Remove outliers which are not within 3 standard deviations from mean of trip dataset
trip17 = trip17[
    np.abs(trip17.WTTRDFIN - trip17.WTTRDFIN.mean()) <= (3*trip17.WTTRDFIN.std())
]

In [ ]:
trip17_1pv = trip17.groupby(['HOUSEID','VEHID'], as_index=False)['VMT_MILE'].sum()

In [ ]:
trip17_process=trip17_1pv.groupby(['HOUSEID']).agg(list)

In [ ]:
trip17_process['freq_car_index']=trip17_process.agg(lambda x: x['VEHID'][np.argmax(x['VMT_MILE'])],axis=1)

In [ ]:
trip17_process=trip17_process.reset_index()

In [ ]:
trip17_process.head()

In [ ]:
trip17.head()

In [ ]:
trip17_final=trip17.merge(trip17_process[['HOUSEID','freq_car_index']],on=['HOUSEID'],how='left')

In [ ]:
trip17_final['selected_car']=trip17_final['freq_car_index']==trip17_final['VEHID']

In [ ]:
trip17_final[['freq_car_index','VEHID','selected_car']].head()

In [ ]:
trip17_final = trip17_final[trip17_final['selected_car'] == True]

In [ ]:
trip17_final.head()

In [ ]:
simu = pd.read_csv('new_simulation_result_income_162500.csv')

In [ ]:
simu.head()

In [ ]:
trip17_pred=trip17_final.merge(simu,on=['HOUSEID'],how='left')

In [ ]:
trip17_pred.head()

In [ ]:
### Trip distance in miles for personally driven vehicle trips
### new Trip Distance Distribution with new categrarical CENSUS DIVISIONS using Dummy Variables
plt.style.use("seaborn")
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }

### MUDs share acrsso the region from AHS

In [ ]:
# 32.6%	34.1%	21.1%	18.6%	21.3%	15.4%	20.6%	19.7%	28.6%
data_share_mud = [.326,.341,.211,.186,.213,.154,.206,.197,.286]

In [ ]:
division_list = ['New England','Middle Atlantic','East North Central','West North Central',
                 'South Atlantic','East South Central','West South Central','Mountain','Pacific']

In [ ]:
ind = np.arange(len(division_list))
plt.figure(figsize =(12,8))

plt.bar(ind,data_share_mud)

plt.xlabel('Census Divisions',fontdict = font)
plt.ylabel("Share",fontdict= font)
plt.xticks(ind,division_list)
plt.title('Multi-Unit Dwellings Share per Census Division', fontsize=24)
#.legend(loc='upper right');
plt.xticks(rotation=45)
#plt.legend(loc='best');
s=data_share_mud 
for i in range(len(s)):
    plt.annotate(str(s[i]), xy=(ind[i],s[i]), ha='center', va='bottom')
    


plt.savefig("MUD_Share.png", dpi=300)
plt.show()

In [ ]:
data_mud_division_income = pd.read_excel('UnitInStructure-v2.xlsx',sheet_name='MUD per DIVISION and INCOME')


In [ ]:
data_mud_division_income.head()

In [ ]:
income_list = data_mud_division_income['Income($)']

In [ ]:
#plt.figure(figsize =(12,8))
pd.concat({
    division_list[i]: data_mud_division_income[division_list[i]] for i in range(9)
}, axis=1).plot.bar(width=0.9,figsize =(16,12))
plt.xticks(np.arange(len(income_list)),income_list)
plt.xlabel('Income Categories ($)',fontdict = font)
plt.xticks(rotation=45)
plt.ylabel("Share",fontdict= font)
plt.legend(loc='best');
plt.title('Share of MUDs per Census Division and Income Level', fontsize=24)

plt.show();

### Location classifications: (Home, Workplace, Public, Driving) Distribution during time of day

By two categaries: MUDs and Other

In [ ]:
### Drop arrows where "VEHID" == "-9"&"-1", which means "Not ascertained" and "Appropriate skip" respectively
trip17_pred_veh_determined = trip17_pred[trip17_pred['VEHID'] != -1]
trip17_pred_veh_determined = trip17_pred_veh_determined[trip17_pred_veh_determined['VEHID'] != -9]


In [ ]:
trip17_pred_veh_determined = trip17_pred_veh_determined[trip17_pred_veh_determined['TRVLCMIN'] > 0]
trip17_pred_veh_determined = trip17_pred_veh_determined[trip17_pred_veh_determined['DWELTIME'] > 0]

In [ ]:
trip17_pred_veh_determined['DWELTIME'] = trip17_pred_veh_determined['DWELTIME']/60

In [ ]:
trip17_pred_veh_determined[['HOUSEID','VEHID']].head()

In [ ]:
type(trip17_pred_veh_determined['WHYTRP1S'])

In [ ]:
### classification of "WHYTRP1S", which is "Trip purpose summary"
trip17_pred_veh_determined['WHYTRP1S_GENERAL'] = np.where(trip17_pred_veh_determined['WHYTRP1S']== 1, 'Home', 
                                             np.where(trip17_pred_veh_determined['WHYTRP1S']== 10, 'Work','Public'))


In [ ]:
### analyze time before the starttime of the first trip, use 
### "WHYFROM", which is "Trip Origin Purpose"
trip17_pred_veh_determined['WHYFROM_GENERAL'] = np.where((trip17_pred_veh_determined['WHYFROM']== 1) | (trip17_pred_veh_determined['WHYFROM']== 2), 'Home', 
                                             np.where((trip17_pred_veh_determined['WHYFROM']== 3) | (trip17_pred_veh_determined['WHYFROM']== 4), 'Work','Public'))



In [ ]:
trip17_pred_veh_determined[['HOUSEID','VEHID','WHYTRP1S_GENERAL','WHYFROM_GENERAL']].head()

In [ ]:
trip17_pred_veh_determined.drop(columns='PERSONID',inplace=True)

In [ ]:
trip17_pred_veh_determined.head()

In [ ]:
trip17_pred_veh_determined.drop_duplicates(subset=['HOUSEID','TDTRPNUM','STRTTIME','ENDTIME',
                                                  ],inplace = True)

In [ ]:
trip17_pred_veh_determined.head(20)

In [ ]:
trip17_pred_veh_determined = trip17_pred_veh_determined[trip17_pred_veh_determined.DWELTIME != -9]

In [ ]:
### pick out the columns needed for location classification during time of day 
### from the whole "trip17_pred_veh_determined" dataframe
trip17_pred_veh_determined_part = trip17_pred_veh_determined[['HOUSEID','VEHID','STRTTIME','ENDTIME','TRVLCMIN','DWELTIME','WHYFROM_GENERAL','WHYTRP1S_GENERAL','MUDs_Predict_OW','CENSUS_D']]
trip17_pred_veh_determined_part = trip17_pred_veh_determined_part.groupby(['HOUSEID','VEHID'], as_index=False).agg(lambda x: list(x))

In [ ]:
type(trip17_pred_veh_determined_part)

In [ ]:
temp = trip17_pred_veh_determined[trip17_pred_veh_determined['HOUSEID'] 
                                      == trip17_pred_veh_determined_part.loc[4,'HOUSEID']]
temp.reset_index(inplace=True)
temp

In [ ]:
(1*100 >= temp.loc[0,'ENDTIME'])

In [ ]:
df = df[df['Col2'].notnull()]

In [ ]:
trip17_pred_veh_determined_part_noNAN = trip17_pred_veh_determined_part_1[trip17_pred_veh_determined_part_1['location_hr_22'].notnull()]


In [ ]:
trip17_pred_veh_determined_part.tail()

In [ ]:
 trip17_pred_veh_determined_part_1.head()

In [ ]:
#aks =['Home']*24

trip17_pred_veh_determined_part_1 = trip17_pred_veh_determined_part.copy()
trip17_pred_veh_determined_part_1.reset_index(inplace=True)

for k in range(len(trip17_pred_veh_determined_part_1)):
    temp = trip17_pred_veh_determined[trip17_pred_veh_determined['HOUSEID'] 
                                      == trip17_pred_veh_determined_part_1.loc[k,'HOUSEID']]
    temp.reset_index(inplace=True)
    for i in range(24):
        for count in range(len(temp)):
            ### trip time in a hour
            if ((temp.loc[count,'STRTTIME']>=i*100) & (temp.loc[count,'ENDTIME']<=(i+1)*100)):
                trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = 'Driving'
                #aks[i]='Driving'
            ### trip time over a hour
            if (temp.loc[count,'STRTTIME']<=i*100):
                trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[0,'WHYFROM_GENERAL']
            & (temp.loc[count,'ENDTIME']>=(i+0)*100)
                #aks[i]='Driving'
            if count+2<=len(temp):
                if ((i*100 >= temp.loc[count,'ENDTIME']) & (i*100 <= temp.loc[count+1,'STRTTIME'])):
                    trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[count,'WHYTRP1S_GENERAL']
                    #aks[i]=temp.loc[count,'WHYTRP1S_GENERAL']
        if (temp.loc[0,'STRTTIME']>i*100):
            trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[0,'WHYFROM_GENERAL']
            #aks[i]=temp.loc[0,'WHYFROM_GENERAL']
        if (temp.tail(1)['ENDTIME'].to_list()[0] < i*100):
            trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[len(temp)-1,'WHYTRP1S_GENERAL']
            #aks[i]=temp.tail(1)['WHYTRP1S_GENERAL'].to_list()[0]
    
for i in tqdm (range (101),  
               desc="Loading…",  
               ascii=False, ncols=75): 
    time.sleep(0.01) 
print("Complete.") 
    


In [ ]:
trip17_pred_veh_determined_part.head()

In [ ]:
trip17_pred_veh_determined_part.head(20)

In [ ]:
100/60*0.6

In [ ]:
#aks =['Home']*24


for k in range(len(trip17_pred_veh_determined_part)):
    temp = trip17_pred_veh_determined[trip17_pred_veh_determined['HOUSEID'] 
                                      == trip17_pred_veh_determined_part.loc[k,'HOUSEID']]
    temp.reset_index(inplace=True)
    
    work = 0
    home = 0
    public = 0
    
    for count in range(len(temp)-1):
        ### trip time in a hour
        if (temp.loc[count,'WHYTRP1S_GENERAL'] == 'Work'):
            #trip17_pred_veh_determined_part.loc[k,'dwell_home'] = 'Driving'
            work += temp.loc[count,'DWELTIME'] # - temp.loc[count,'ENDTIME']
            #work += temp.loc[count+1,'STRTTIME'] - temp.loc[count,'ENDTIME']
            #aks[i]='Driving'
        ### trip time over a hour
        if (temp.loc[count,'WHYTRP1S_GENERAL'] == 'Home'):
            #trip17_pred_veh_determined_part.loc[k,'dwell_home'] = 'Driving'
            home += temp.loc[count,'DWELTIME']
            #home += temp.loc[count+1,'STRTTIME'] - temp.loc[count,'ENDTIME']
        if (temp.loc[count,'WHYTRP1S_GENERAL'] == 'Public'):
            #trip17_pred_veh_determined_part.loc[k,'dwell_home'] = 'Driving'
            public += temp.loc[count,'DWELTIME']
            #public += temp.loc[count+1,'STRTTIME'] - temp.loc[count,'ENDTIME']
#         if ((temp.loc[count,'STRTTIME']<=i*100) & (temp.loc[count,'ENDTIME']>=(i+0)*100)):
#             trip17_pred_veh_determined_part.loc[k,'dwell_work'] = 'Driving'
#             #aks[i]='Driving'
#         if count+2<=len(temp):
#             if ((i*100 >= temp.loc[count,'ENDTIME']) & (i*100 <= temp.loc[count+1,'STRTTIME'])):
#                 trip17_pred_veh_determined_part.loc[k,'dwell_public'] = temp.loc[count,'WHYTRP1S_GENERAL']
#                 #aks[i]=temp.loc[count,'WHYTRP1S_GENERAL']
    if (temp.loc[0,'WHYFROM_GENERAL'] == 'Work'):
        work += temp.loc[0,'STRTTIME']/60*0.6
    if (temp.loc[0,'WHYFROM_GENERAL'] == 'Home'):
        home += temp.loc[0,'STRTTIME']/60*0.6
    if (temp.loc[0,'WHYFROM_GENERAL'] == 'Public'):
        public += temp.loc[0,'STRTTIME']/60*0.6
    
        #trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[0,'WHYFROM_GENERAL']
        #aks[i]=temp.loc[0,'WHYFROM_GENERAL']
    if temp.tail(1)['ENDTIME'].to_list()[0] < 2400:
        if (temp.tail(1)['WHYTRP1S_GENERAL'].to_list()[0] == 'Work'):
            work += temp.tail(1)['DWELTIME'].to_list()[0]#2400 - temp.tail(1)['STRTTIME'].to_list()[0] - temp.tail(1)['TRVLCMIN'].to_list()[0]
        if (temp.tail(1)['WHYTRP1S_GENERAL'].to_list()[0] == 'Home'):
            home += temp.tail(1)['DWELTIME'].to_list()[0]#2400 - temp.tail(1)['STRTTIME'].to_list()[0] - temp.tail(1)['TRVLCMIN'].to_list()[0]
        if (temp.tail(1)['WHYTRP1S_GENERAL'].to_list()[0] == 'Public'):
            public += temp.tail(1)['DWELTIME'].to_list()[0]#2400 - temp.tail(1)['STRTTIME'].to_list()[0] - temp.tail(1)['TRVLCMIN'].to_list()[0]
#         trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[len(temp)-1,'WHYTRP1S_GENERAL']
        #aks[i]=temp.tail(1)['WHYTRP1S_GENERAL'].to_list()[0]
    trip17_pred_veh_determined_part.loc[k,'dwell_home'] = home
    trip17_pred_veh_determined_part.loc[k,'dwell_work'] = work
    trip17_pred_veh_determined_part.loc[k,'dwell_public'] = public
    
for i in tqdm (range (101),  
               desc="Loading…",  
               ascii=False, ncols=75): 
    time.sleep(0.01) 
print("Complete.") 
    



In [ ]:
trip17_pred_veh_determined.head(10)

In [ ]:
temp = trip17_pred_veh_determined[trip17_pred_veh_determined['HOUSEID'] 
                                      == trip17_pred_veh_determined_part.loc[2,'HOUSEID']]

In [ ]:
temp

In [ ]:
#aks =['Home']*24


for k in range(len(trip17_pred_veh_determined_part)):
    temp = trip17_pred_veh_determined[trip17_pred_veh_determined['HOUSEID'] 
                                      == trip17_pred_veh_determined_part.loc[k,'HOUSEID']]
    temp.reset_index(inplace=True)
    
    work = 0
    home = 0
    public = 0
    
    for count in range(len(temp)-1):
        ### trip time in a hour
        if (temp.loc[count,'WHYTRP1S_GENERAL'] == 'Work'):
            #trip17_pred_veh_determined_part.loc[k,'dwell_home'] = 'Driving'
            work += temp.loc[count+1,'STRTTIME'] - temp.loc[count,'ENDTIME']
            #aks[i]='Driving'
        ### trip time over a hour
        if (temp.loc[count,'WHYTRP1S_GENERAL'] == 'Home'):
            #trip17_pred_veh_determined_part.loc[k,'dwell_home'] = 'Driving'
            home += temp.loc[count+1,'STRTTIME'] - temp.loc[count,'ENDTIME']
        if (temp.loc[count,'WHYTRP1S_GENERAL'] == 'Public'):
            #trip17_pred_veh_determined_part.loc[k,'dwell_home'] = 'Driving'
            public += temp.loc[count+1,'STRTTIME'] - temp.loc[count,'ENDTIME']
#         if ((temp.loc[count,'STRTTIME']<=i*100) & (temp.loc[count,'ENDTIME']>=(i+0)*100)):
#             trip17_pred_veh_determined_part.loc[k,'dwell_work'] = 'Driving'
#             #aks[i]='Driving'
#         if count+2<=len(temp):
#             if ((i*100 >= temp.loc[count,'ENDTIME']) & (i*100 <= temp.loc[count+1,'STRTTIME'])):
#                 trip17_pred_veh_determined_part.loc[k,'dwell_public'] = temp.loc[count,'WHYTRP1S_GENERAL']
#                 #aks[i]=temp.loc[count,'WHYTRP1S_GENERAL']
    if (temp.loc[0,'WHYFROM_GENERAL'] == 'Work'):
        work += temp.loc[0,'STRTTIME']
    if (temp.loc[0,'WHYFROM_GENERAL'] == 'Home'):
        home += temp.loc[0,'STRTTIME']
    if (temp.loc[0,'WHYFROM_GENERAL'] == 'Public'):
        public += temp.loc[0,'STRTTIME']
    
        #trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[0,'WHYFROM_GENERAL']
        #aks[i]=temp.loc[0,'WHYFROM_GENERAL']
    if temp.tail(1)['ENDTIME'].to_list()[0] < 2400:
        if (temp.tail(1)['WHYTRP1S_GENERAL'].to_list()[0] == 'Work'):
            work += 2400 - temp.tail(1)['STRTTIME'].to_list()[0] - temp.tail(1)['TRVLCMIN'].to_list()[0]
        if (temp.tail(1)['WHYTRP1S_GENERAL'].to_list()[0] == 'Home'):
            home += 2400 - temp.tail(1)['STRTTIME'].to_list()[0] - temp.tail(1)['TRVLCMIN'].to_list()[0]
        if (temp.tail(1)['WHYTRP1S_GENERAL'].to_list()[0] == 'Public'):
            public += 2400 - temp.tail(1)['STRTTIME'].to_list()[0] - temp.tail(1)['TRVLCMIN'].to_list()[0]
#         trip17_pred_veh_determined_part_1.loc[k,'location_hr_'+str(i+1)] = temp.loc[len(temp)-1,'WHYTRP1S_GENERAL']
        #aks[i]=temp.tail(1)['WHYTRP1S_GENERAL'].to_list()[0]
    trip17_pred_veh_determined_part.loc[k,'dwell_home'] = home
    trip17_pred_veh_determined_part.loc[k,'dwell_work'] = work
    trip17_pred_veh_determined_part.loc[k,'dwell_public'] = public
    
for i in tqdm (range (101),  
               desc="Loading…",  
               ascii=False, ncols=75): 
    time.sleep(0.01) 
print("Complete.") 
    




In [ ]:
dwell_process = trip17_pred_veh_determined_part
#dwell_process = trip17_pred_veh_determined[trip17_pred_veh_determined['DWELTIME'] != -9]
dwell_process.head()

In [ ]:
mask = dwell_process.MUDs_Predict_OW.apply(lambda x: 1.0 in x)
dwell_process_mud = dwell_process[mask]

In [ ]:
mask = dwell_process.MUDs_Predict_OW.apply(lambda x: 0.0 in x)
dwell_process_other = dwell_process[mask]

In [ ]:
dwell_process[dwell_process['dwell_home'] > 0].head(2)

In [ ]:
dwell_process = dwell_process[dwell_process['dwell_home'] > 0]
dwell_process = dwell_process[dwell_process['dwell_work'] > 0]
dwell_process = dwell_process[dwell_process['dwell_public'] > 0]

In [ ]:
dwell_process_home = dwell_process['dwell_home']
dwell_process_work = dwell_process['dwell_work']
dwell_process_public = dwell_process['dwell_public']

In [ ]:
dwell_process_mud_home = dwell_process_mud['dwell_home']
dwell_process_mud_work = dwell_process_mud['dwell_work']
dwell_process_mud_public = dwell_process_mud['dwell_public']

dwell_process_other_home = dwell_process_other['dwell_home']
dwell_process_other_work = dwell_process_other['dwell_work']
dwell_process_other_public = dwell_process_other['dwell_public']

In [ ]:
# dwell_process_mud_home = dwell_process_mud[dwell_process_mud['WHYTRP1S_GENERAL'] == 'Home']
# dwell_process_mud_work = dwell_process_mud[dwell_process_mud['WHYTRP1S_GENERAL'] == 'Work']
# dwell_process_mud_public = dwell_process_mud[dwell_process_mud['WHYTRP1S_GENERAL'] == 'Public']

# dwell_process_other_home = dwell_process_other[dwell_process_other['WHYTRP1S_GENERAL'] == 'Home']
# dwell_process_other_work = dwell_process_other[dwell_process_other['WHYTRP1S_GENERAL'] == 'Work']
# dwell_process_other_public = dwell_process_other[dwell_process_other['WHYTRP1S_GENERAL'] == 'Public']

In [ ]:
fig = plt.figure(figsize = (16,8))
p1=sns.kdeplot(dwell_process_home, shade=True, label = 'Home',color = '#0000FF')
p1=sns.kdeplot(dwell_process_work, shade=True,label = 'Workplace',color = '#FF0000')
p1=sns.kdeplot(dwell_process_public, shade=True,label = 'Public',color ='#008000')
p1=sns.kdeplot(dwell_process_other_home*0.6, shade=True,label = 'Home of others',color = '#87CEFA',linestyle="--")
p1=sns.kdeplot(dwell_process_other_work*0.6, shade=True,label = 'Workplace of others',color ='#FFA07A',linestyle="--")
p1=sns.kdeplot(dwell_process_other_public*0.6, shade=True, label = 'Public of Others',color = '#7CFC00',linestyle="--")
plt.title('Distribution of Dwell Time', fontsize=28)
plt.xlabel('Hours (Hrs)',fontdict=font);
plt.ylabel('Density',fontdict=font);
plt.xlim(-300/60, 2400/60) 
# plt.xticle()
plt.legend(loc='best')
plt.show();

fig.savefig('dist_dwell.png',dpi=1200)

In [ ]:
fig = plt.figure(figsize = (16,8))
p1=sns.kdeplot(dwell_process_home, shade=True, label = 'Home',color = '#0000FF')
p1=sns.kdeplot(dwell_process_work, shade=True,label = 'Workplace',color = '#FF0000')
p1=sns.kdeplot(dwell_process_public, shade=True,label = 'Public',color ='#008000')
# p1=sns.kdeplot(dwell_process_other_home*0.6, shade=True,label = 'Home of others',color = '#87CEFA',linestyle="--")
# p1=sns.kdeplot(dwell_process_other_work*0.6, shade=True,label = 'Workplace of others',color ='#FFA07A',linestyle="--")
# p1=sns.kdeplot(dwell_process_other_public*0.6, shade=True, label = 'Public of Others',color = '#7CFC00',linestyle="--")
plt.title('Distribution of Dwell Time', fontsize=28)
plt.xlabel('Hours (Hrs)',fontdict=font);
plt.ylabel('Density',fontdict=font);
plt.xlim(-300/60, 2400/60) 
# plt.xticle()
plt.legend(loc='best')
plt.show();

fig.savefig('dist_dwell.png',dpi=1200)

Smooth density plot

In [ ]:

dwell_process = trip17_pred_veh_determined_part_noNAN[['HOUSEID','VEHID','MUDs_Predict_OW',
                                                  'location_hr_1','location_hr_2','location_hr_3','location_hr_4',
                                                  'location_hr_5','location_hr_6','location_hr_7','location_hr_8',
                                                  'location_hr_9','location_hr_10','location_hr_11','location_hr_12',
                                                  'location_hr_13','location_hr_14','location_hr_15',
                                                  'location_hr_16','location_hr_17','location_hr_18',
                                                  'location_hr_19','location_hr_20','location_hr_21',
                                                  'location_hr_22','location_hr_23','location_hr_24']]

In [ ]:
for i in range(len(dwell_process)):
    dwell_process.loc[i,'DWELL_HOME'] = sum(dwell_process.iloc[i,:]=='Home')

In [ ]:
for i in range(len(dwell_process)):
    dwell_process.loc[i,'DWELL_WORK'] = sum(dwell_process.iloc[i,:]=='Work')

In [ ]:
for i in range(len(dwell_process)):
    dwell_process.loc[i,'DWELL_PUBLIC'] = sum(dwell_process.iloc[i,:]=='Public')

In [ ]:
dwell_process.head()

In [ ]:
import random

In [ ]:
for i in range(len(dwell_process)):
    dwell_process.loc[i,'DWELL_WORK'] = dwell_process.loc[i,'DWELL_WORK']  random.uniform(0, 0.5)

In [ ]:
mask = dwell_process.MUDs_Predict_OW.apply(lambda x: 1.0 in x)
dwell_process_mud = dwell_process[mask]

mask = dwell_process.MUDs_Predict_OW.apply(lambda x: 0.0 in x)
dwell_process_other = dwell_process[mask]

In [ ]:
dwell_process_mud_home = dwell_process_mud['DWELL_HOME']
dwell_process_mud_work = dwell_process_mud['DWELL_WORK']
dwell_process_mud_public = dwell_process_mud['DWELL_PUBLIC']

dwell_process_other_home = dwell_process_other['DWELL_HOME']
dwell_process_other_work = dwell_process_other['DWELL_WORK']
dwell_process_other_public = dwell_process_other['DWELL_PUBLIC']

In [ ]:
fig = plt.figure(figsize = (16,8), dpi=300)
p1=sns.kdeplot(dwell_process_mud_home, shade=True, label = 'Home of MUDs',color = '#0000FF')
p1=sns.kdeplot(dwell_process_mud_work, shade=True,label = 'Workplace of MUDs',color = '#FF0000')
p1=sns.kdeplot(dwell_process_mud_public, shade=True,label = 'Public of MUDs',color ='#008000')
p1=sns.kdeplot(dwell_process_other_home, shade=True,label = 'Home of others',color = '#87CEFA',linestyle="--")
p1=sns.kdeplot(dwell_process_other_work, shade=True,label = 'Workplace of others',color ='#FFA07A',linestyle="--")
p1=sns.kdeplot(dwell_process_other_public, shade=True, label = 'Public of Others',color = '#7CFC00',linestyle="--")
plt.title('Distribution of Dwell Time', fontsize=28)
plt.xlabel('Times (hrs)',fontdict=font);
plt.ylabel('Density',fontdict=font);
plt.legend(loc='best')
plt.show();

In [ ]:
dwell_process = trip17_pred_veh_determined_part_1[['HOUSEID','VEHID','MUDs_Predict_OW',
                                                  'location_hr_1','location_hr_2','location_hr_3','location_hr_4',
                                                  'location_hr_5','location_hr_6','location_hr_7','location_hr_8',
                                                  'location_hr_9','location_hr_10','location_hr_11','location_hr_12',
                                                  'location_hr_13','location_hr_14','location_hr_15',
                                                  'location_hr_16','location_hr_17','location_hr_18',
                                                  'location_hr_19','location_hr_20','location_hr_21',
                                                  'location_hr_22','location_hr_23','location_hr_24']]

In [ ]:
dwell_process.head()

In [ ]:
dwell_process.iloc[0,:]=='Home'

In [ ]:
for i in range(len(dwell_process)):
    dwell_process.loc[i,'DWELL_HOME'] = sum(dwell_process.iloc[i,:]=='Home')

In [ ]:
for i in range(len(dwell_process)):
    dwell_process.loc[i,'DWELL_WORK'] = sum(dwell_process.iloc[i,:]=='Work')

In [ ]:
for i in range(len(dwell_process)):
    dwell_process.loc[i,'DWELL_PUBLIC'] = sum(dwell_process.iloc[i,:]=='Public')

In [ ]:
dwell_process.head(10)

In [ ]:
# dwell_process['DWELL_HOME'] = dwell_process[
# dwell_process['DWELL_WORK'] = 
# dwell_process['DWELL_PUBLIC'] = 

In [ ]:
dwell_process = trip17_pred_veh_determined[trip17_pred_veh_determined['DWELTIME'] != -9]
dwell_process = dwell_process[['HOUSEID','VEHID','DWELTIME','WHYFROM_GENERAL','WHYTRP1S_GENERAL','MUDs_Predict_OW']]

In [ ]:
dwell_process.head()

In [ ]:
mask = dwell_process.MUDs_Predict_OW.apply(lambda x: 1.0 in x)
dwell_process_mud = dwell_process[mask]

In [ ]:
mask = dwell_process.MUDs_Predict_OW.apply(lambda x: 0.0 in x)
dwell_process_other = dwell_process[mask]

In [ ]:
dwell_process_other.head()

In [ ]:
dwell_process_mud_home = dwell_process_mud['DWELL_HOME']
dwell_process_mud_work = dwell_process_mud['DWELL_WORK']
dwell_process_mud_public = dwell_process_mud['DWELL_PUBLIC']

dwell_process_other_home = dwell_process_other['DWELL_HOME']
dwell_process_other_work = dwell_process_other['DWELL_WORK']
dwell_process_other_public = dwell_process_other['DWELL_PUBLIC']

In [ ]:
dwell_process_mud_home = dwell_process_mud[dwell_process_mud['WHYTRP1S_GENERAL'] == 'Home']
dwell_process_mud_work = dwell_process_mud[dwell_process_mud['WHYTRP1S_GENERAL'] == 'Work']
dwell_process_mud_public = dwell_process_mud[dwell_process_mud['WHYTRP1S_GENERAL'] == 'Public']

dwell_process_other_home = dwell_process_other[dwell_process_other['WHYTRP1S_GENERAL'] == 'Home']
dwell_process_other_work = dwell_process_other[dwell_process_other['WHYTRP1S_GENERAL'] == 'Work']
dwell_process_other_public = dwell_process_other[dwell_process_other['WHYTRP1S_GENERAL'] == 'Public']

In [ ]:
dwell_process_mud_home.head()

In [ ]:
dwell_process_other_public

In [ ]:
fig = plt.figure(figsize = (16,8))
p1=sns.kdeplot(dwell_process_mud_home, shade=True, label = 'Home of MUDs',color = '#0000FF')
p1=sns.kdeplot(dwell_process_mud_work, shade=True,label = 'Workplace of MUDs',color = '#FF0000')
p1=sns.kdeplot(dwell_process_mud_public, shade=True,label = 'Public of MUDs',color ='#008000')
p1=sns.kdeplot(dwell_process_other_home, shade=True,label = 'Home of others',color = '#87CEFA',linestyle="--")
p1=sns.kdeplot(dwell_process_other_work, shade=True,label = 'Workplace of others',color ='#FFA07A',linestyle="--")
p1=sns.kdeplot(dwell_process_other_public, shade=True, label = 'Public of Others',color = '#7CFC00',linestyle="--")
plt.title('Distribution of Dwell Time', fontsize=28)
plt.xlabel('Times (hrs)',fontdict=font);
plt.ylabel('Density',fontdict=font);
plt.legend(loc='best')
plt.show();

In [ ]:
trip17_pred_veh_determined_part_1

In [ ]:
temp = trip17_pred_veh_determined[trip17_pred_veh_determined['HOUSEID'] == trip17_pred_veh_determined_part.loc[0,'HOUSEID']]
#trip17_pred_veh_determined_part.loc[0,'location_hr_1'] == 'Home'
temp
i=0
while(temp.loc[0,'STRTTIME']>i*100):
    trip17_pred_veh_determined_part.loc[0,'location_hr_'+str(i+1)] = temp.loc[0,'WHYFROM_GENERAL']
    i+=1

j=23
while(temp.tail(1)['ENDTIME'].to_list()[0] < j*100):
    trip17_pred_veh_determined_part.loc[0,'location_hr_'+str(j-1)] = temp.loc[0,'WHYTRP1S_GENERAL']
    j-=1

k=i
while((temp.loc[0,'STRTTIME']>k*100) & (temp.loc[0,'ENDTIME']<k*100)):
    trip17_pred_veh_determined_part.loc[0,'location_hr_'+str(k+1)] = 'Driving'
    k+=1


In [ ]:
temp.tail(1)['ENDTIME'].to_list()[0]

In [ ]:
lst = trip17_pred_veh_determined_part['STRTTIME']

In [ ]:
### assign 24 hrs location classification for each household to 24 columns
trip17_pred_veh_determined_part['location_hr_1'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 100,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 0,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 100,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_2'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 200,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 100,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 200,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))



In [ ]:
trip17_pred_veh_determined_part['location_hr_3'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 300,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 200,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 300,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))



In [ ]:
trip17_pred_veh_determined_part['location_hr_4'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 400,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 300,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 400,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))



In [ ]:
trip17_pred_veh_determined_part['location_hr_5'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 500,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 400,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 500,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))



In [ ]:
trip17_pred_veh_determined_part['location_hr_6'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 600,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 500,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 600,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))







In [ ]:
trip17_pred_veh_determined_part['location_hr_7'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 700,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 600,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 700,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))






In [ ]:
trip17_pred_veh_determined_part['location_hr_8'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 800,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 700,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 800,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))





In [ ]:
trip17_pred_veh_determined_part['location_hr_9'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 900,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 800,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 900,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))




In [ ]:
trip17_pred_veh_determined_part['location_hr_10'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1000,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 900,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 1000,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))



In [ ]:
trip17_pred_veh_determined_part['location_hr_11'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1100,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1000,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 1100,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_12'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1200,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1100,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 1200,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))



In [ ]:
trip17_pred_veh_determined_part['location_hr_13'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1300,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1200,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 1300,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_14'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1400,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1300,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 1400,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_15'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1500,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1400,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 1500,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_16'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1600,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1500,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 1600,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_17'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1700,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1600,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 1700,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_18'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1800,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1700,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 1800,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_19'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1900,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1800,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 1900,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_20'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 2000,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 1900,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 2000,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))



In [ ]:
trip17_pred_veh_determined_part['location_hr_21'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 2100,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 2000,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 2100,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_22'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 2200,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 2100,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 2200,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))




In [ ]:
trip17_pred_veh_determined_part['location_hr_23'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 2300,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 2200,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 2300,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_24'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 2400,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[0])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[0])) > 2300,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) > 2400,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])))))


In [ ]:
trip17_pred_veh_determined_part['location_hr_10'] = np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[0])) < 900,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[0])), 
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[1])) > 1000,
                                                            pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYFROM_GENERAL']))[1])),
                                                            np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['STRTTIME']))[1])) > 900,'Driving',
                                                                     np.where(pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['ENDTIME']))[1])) > 1000,'Driving',
                                                                              pd.Series(list(list(itertools.zip_longest(*trip17_pred_veh_determined_part['WHYTRP1S_GENERAL']))[1]))))))



In [ ]:
trip17_pred_veh_determined_part.head(4)

In [ ]:
len(trip17_pred_veh_determined_part)

In [ ]:
trip17_pred_veh_determined_part_mud = trip17_pred_veh_determined_part[pd.Series(list(list(
    itertools.zip_longest(*trip17_pred_veh_determined_part['MUDs_Predict_OW']))[0])) == 1.0]
trip17_pred_veh_determined_part_other = trip17_pred_veh_determined_part[pd.Series(list(list(
    itertools.zip_longest(*trip17_pred_veh_determined_part['MUDs_Predict_OW']))[0])) == 0.0]

In [ ]:
len(trip17_pred_veh_determined_part_other)+len(trip17_pred_veh_determined_part_mud)

In [ ]:
home_share_mud = []
for i in range(24):
    home_mud = len(trip17_pred_veh_determined_part_mud[trip17_pred_veh_determined_part_mud['location_hr_'+str(i+1)]
                                                       == 'Work'])/len(trip17_pred_veh_determined_part_mud)
    home_share_mud.append(home_mud)

In [ ]:
trip17_pred_veh_determined_part_mud['location_hr_1'].value_counts()

In [ ]:
len(trip17_pred_veh_determined_part_mud[trip17_pred_veh_determined_part_mud['location_hr_'+str(0+1)]
                                                       == 'Home'])

In [ ]:
df.head()

In [ ]:
df = pd.DataFrame(columns=['home_share_mud', 'work_share_mud', 'public_share_mud', 'driving_share_mud'])
df['home_share_mud'] = home_share_mud
df['work_share_mud'] = work_share_mud
df['public_share_mud'] = public_share_mud
df['driving_share_mud'] = driving_share_mud
df.head()

In [ ]:
trip17_pred_veh_determined_part_1.head(20)

In [ ]:
trip17_pred_veh_determined_part_mud = trip17_pred_veh_determined_part_1[pd.Series(list(list(
    itertools.zip_longest(*trip17_pred_veh_determined_part_1['MUDs_Predict_OW']))[0])) == 1.0]
trip17_pred_veh_determined_part_other = trip17_pred_veh_determined_part_1[pd.Series(list(list(
    itertools.zip_longest(*trip17_pred_veh_determined_part_1['MUDs_Predict_OW']))[0])) == 0.0]

In [ ]:
labels = []
for i in range(24):
    labels.append(str(i) + ':00 to ' + str(i+1) + ':00')
home_share_mud = []
work_share_mud = []
public_share_mud = []
driving_share_mud = []
home_share_other = []
work_share_other = []
public_share_other = []
driving_share_other = []
for i in range(24):
    home_mud = len(trip17_pred_veh_determined_part_mud[trip17_pred_veh_determined_part_mud['location_hr_'+str(i+1)]
                                                       == 'Home'])/len(trip17_pred_veh_determined_part_mud)
    home_share_mud.append(home_mud)
    work_mud = len(trip17_pred_veh_determined_part_mud[trip17_pred_veh_determined_part_mud['location_hr_'+str(i+1)]
                                                       == 'Work'])/len(trip17_pred_veh_determined_part_mud)
    work_share_mud.append(work_mud)
    public_mud = len(trip17_pred_veh_determined_part_mud[trip17_pred_veh_determined_part_mud['location_hr_'+str(i+1)]
                                                       == 'Public'])/len(trip17_pred_veh_determined_part_mud)
    public_share_mud.append(public_mud)
    driving_mud = len(trip17_pred_veh_determined_part_mud[trip17_pred_veh_determined_part_mud['location_hr_'+str(i+1)]
                                                       == 'Driving'])/len(trip17_pred_veh_determined_part_mud)
    driving_share_mud.append(driving_mud)
    
#     home_other = len(trip17_pred_veh_determined_part_other[trip17_pred_veh_determined_part_other['location_hr_'+str(i+1)]
#                                                        == 'Home'])/len(trip17_pred_veh_determined_part_other)
#     home_share_other.append(home_other)
    work_other = len(trip17_pred_veh_determined_part_other[trip17_pred_veh_determined_part_other['location_hr_'+str(i+1)]
                                                       == 'Work'])/len(trip17_pred_veh_determined_part_other)
    work_share_other.append(work_other)
    public_other = len(trip17_pred_veh_determined_part_other[trip17_pred_veh_determined_part_other['location_hr_'+str(i+1)]
                                                       == 'Public'])/len(trip17_pred_veh_determined_part_other)
    public_share_other.append(public_other)
    driving_other = len(trip17_pred_veh_determined_part_other[trip17_pred_veh_determined_part_other['location_hr_'+str(i+1)]
                                                       == 'Driving'])/len(trip17_pred_veh_determined_part_other)
    driving_share_other.append(driving_other)
    home_other = 1-work_other-public_other-driving_other
    home_share_other.append(home_other)
    
bar12 = np.add(home_share_mud, driving_share_mud).tolist()
bar123 = np.add(bar12, work_share_mud).tolist()

bar12other = np.add(home_share_other, driving_share_other).tolist()
bar123other = np.add(bar12other, work_share_other).tolist()
    
x = np.arange(len(labels))  # the label locations
width = 0.35       # the width of the bars: can also be len(x) sequence


fig = plt.figure(figsize=(16, 8))

plt.bar(x-width/2, home_share_mud, width, label='Home of MUDs',color='#00008B')
plt.bar(x-width/2, driving_share_mud, width, bottom=home_share_mud, label='Driving of MUDs',color='#0000FF')
plt.bar(x-width/2, work_share_mud, width, bottom=bar12, label='Work of MUDs',color='#00BFFF')
plt.bar(x-width/2, public_share_mud, width, bottom=bar123, label='Public of MUDs',color='#B0E0E6')

plt.bar(x+width/2, home_share_other, width, label='Home of others',color='#8B0000')
plt.bar(x+width/2, driving_share_other, width, bottom=home_share_other, label='Driving of others',color='#FF0000')
plt.bar(x+width/2, work_share_other, width, bottom=bar12other, label='Work of others',color='#F08080')
plt.bar(x+width/2, public_share_other, width, bottom=bar123other, label='Public of others',color='#FFA07A')


#df.plot.bar(ax=axes[1], stacked=True)

plt.xticks(Rotation=45) 
plt.xlabel('Time of Day (hrs)',fontdict=font)
plt.ylabel('% of Vehicles',fontdict=font)
plt.title('Percentage of vehicles per time of day',fontsize=24)
plt.legend(loc='upper left', bbox_to_anchor=(0.95, 1.025))

plt.show()

In [ ]:
labels = ['G1', 'G2', 'G3', 'G4', 'G5']
men_means = [20, 34, 30, 35, 27]
women_means = [25, 32, 34, 20, 25]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(16,10))
rects1 = ax.bar(x - width/2, men_means, width, label='Men')
rects2 = ax.bar(x + width/2, women_means, width, label='Women')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()

### Variation of VMT per time of day for 100 scenarios

In [ ]:
trip17_vmt = trip17_pred[trip17_pred['VMT_MILE']>0]
trip17_vmt = trip17_vmt[trip17_vmt['TRVLCMIN']>0]


In [ ]:
trip17_vmt_T = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 1]
trip17_vmt_F = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 0]

for i in range(24):
    trip17_vmt_T['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_T['STRTTIME'] >= 100*i) & (trip17_vmt_T['STRTTIME']< 100*(i+1)), 
                                               trip17_vmt_T['VMT_MILE']/(trip17_vmt_T['TRVLCMIN']/60),0)
    trip17_vmt_T['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_T['STRTTIME'] >= 100*i) & (trip17_vmt_T['ENDTIME']< 100*(i+1)), 
                                               trip17_vmt_T['VMT_MILE']/(trip17_vmt_T['TRVLCMIN']/60),0)
    trip17_vmt_F['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_F['STRTTIME'] >= 100*i) & (trip17_vmt_F['STRTTIME']< 100*(i+1)), 
                                               trip17_vmt_F['VMT_MILE']/(trip17_vmt_F['TRVLCMIN']/60),0)
    trip17_vmt_F['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_F['STRTTIME'] >= 100*i) & (trip17_vmt_F['ENDTIME']< 100*(i+1)), 
                                               trip17_vmt_F['VMT_MILE']/(trip17_vmt_F['TRVLCMIN']/60),0)


In [ ]:
trip17_vmt_T.head()

In [ ]:
avg_vmt_T = [[0 for i in range(24)] for j in range(100)]
avg_vmt_F = [[0 for i in range(24)] for j in range(100)]
avg_vmt = [[0 for i in range(24)] for j in range(100)]

# trip17_vmt_T = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 1]
# trip17_vmt_F = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 0]
trip17_vmt_T = trip17_vmt_T[trip17_vmt_T['MUDs_Predict_OW'] == 1]
trip17_vmt_F = trip17_vmt_F[trip17_vmt_F['MUDs_Predict_OW'] == 0]


#trip17_MUD_C['HHIncomes'].mask(trip17_MUD_C['HHFAMINC'] == i+1, income[i], inplace=True)
for i in range(24):
    #avg_vmt_T[0][i] = trip17_vmt_T[(trip17_vmt_T['STRTTIME'] >= 100*i) & (trip17_vmt_T['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    avg_vmt_T[0][i] = trip17_vmt_T['VMT_Hr'+str(i+1)].mean()
    #avg_vmt_F[0][i] = trip17_vmt_F[(trip17_vmt_F['STRTTIME'] >= 100*i) & (trip17_vmt_F['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    avg_vmt_F[0][i] = trip17_vmt_F['VMT_Hr'+str(i+1)].mean()
    avg_vmt[0][i] = trip17_vmt[(trip17_vmt['STRTTIME'] >= 100*i) & (trip17_vmt['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    

In [ ]:
for i in range(1,100):
#     trip17_vmt_T = trip17_vmt[trip17_vmt['MUDs_Predict_OW_'+str(i)] == 1]
#     trip17_vmt_F = trip17_vmt[trip17_vmt['MUDs_Predict_OW_'+str(i)] == 0]
    trip17_vmt_T = trip17_vmt_T[trip17_vmt_T['MUDs_Predict_OW_'+str(i)] == 1]
    trip17_vmt_F = trip17_vmt_F[trip17_vmt_F['MUDs_Predict_OW_'+str(i)] == 0]
    for j in range(24):
#         avg_vmt_T[i][j] = trip17_vmt_T[(trip17_vmt_T['STRTTIME'] >= 100*j) & (trip17_vmt_T['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        avg_vmt_T[i][j] = trip17_vmt_T['VMT_Hr'+str(j+1)].mean()
#         avg_vmt_F[i][j] = trip17_vmt_F[(trip17_vmt_F['STRTTIME'] >= 100*j) & (trip17_vmt_F['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        avg_vmt_F[i][j] = trip17_vmt_F['VMT_Hr'+str(j+1)].mean()
        avg_vmt[i][j] = trip17_vmt[(trip17_vmt['STRTTIME'] >= 100*j) & (trip17_vmt['STRTTIME']< 100*(j+1))].VMT_MILE.mean()

# for i in range(24):
#     trip17_vmt_T.loc[trip17_vmt_T['VMT_Hr'+str(i+1)] > 120,'VMT_Hr'+str(i+1)] = 120
#     trip17_vmt_F.loc[trip17_vmt_F['VMT_Hr'+str(i+1)] > 120,'VMT_Hr'+str(i+1)] = 120

In [ ]:
col_list = range(24)
df_avg_vmt_T = pd.DataFrame(data=avg_vmt_T, columns=col_list)
df_avg_vmt_F = pd.DataFrame(data=avg_vmt_F, columns=col_list)
df_avg_vmt = pd.DataFrame(data=avg_vmt, columns=col_list)
df_avg_vmt_T

In [ ]:
df_avg_vmt_T.fillna(0, inplace=True)
df_avg_vmt_F.fillna(0, inplace=True)
df_avg_vmt.fillna(0,inplace=True)

In [ ]:
# function for setting the colors of the box plots pairs
def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

In [ ]:
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')
  

In [ ]:
plt.figure(figsize=(16, 8))
plt.title('Time of Day Average VMT for personally driven vehicle trips (Miles)', fontsize=28)
 
# data required for plotting
all_data = [df_avg_vmt_T[i] for i in range(24)]
all_data_F = [df_avg_vmt_F[i] for i in range(24)]

# plot box plot
bp2 = plt.boxplot(all_data,positions=np.array(range(1,len(all_data)+1))*2-0.3)
set_box_color(bp2, '#0000FF')


# plot box plot
bp4 = plt.boxplot(all_data_F,positions=np.array(range(1,len(all_data)+1))*2+0.3)
#axs[1][1].set_title('Box plot')
set_box_color(bp4, '#FF0000')


plt.plot([], c='#0000FF', label="MUDs' Residents")
plt.plot([], c='#FF0000', label="Others' Residents")
plt.legend()
# adding horizontal grid lines

plt.xticks(range(1, len(tod)*2+1, 2), tod)
plt.xticks(rotation=45)        
plt.xlabel('Time of Day',fontdict=font)
plt.ylabel('VMT(Miles)',fontdict=font)

plt.show()

In [ ]:
trip17_vmt_T = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 1]
trip17_vmt_F = trip17_vmt[trip17_vmt['MUDs_Predict_OW'] == 0]
#trip17_vmt = trip17_vmt[trip17_vmt['TRVLCMIN']>0]


In [ ]:
len(trip17_vmt_F)

In [ ]:
len(trip17_vmt)

In [ ]:
trip17_vmt_T.head()

In [ ]:
for i in range(24):
    trip17_vmt_T['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_T['STRTTIME'] >= 100*i) & (trip17_vmt_T['STRTTIME']< 100*(i+1)), 
                                               trip17_vmt_T['VMT_MILE']/(trip17_vmt_T['TRVLCMIN']/60),0)
    trip17_vmt_T['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_T['STRTTIME'] >= 100*i) & (trip17_vmt_T['ENDTIME']< 100*(i+1)), 
                                               trip17_vmt_T['VMT_MILE']/(trip17_vmt_T['TRVLCMIN']/60),0)
    trip17_vmt_F['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_F['STRTTIME'] >= 100*i) & (trip17_vmt_F['STRTTIME']< 100*(i+1)), 
                                               trip17_vmt_F['VMT_MILE']/(trip17_vmt_F['TRVLCMIN']/60),0)
    trip17_vmt_F['VMT_Hr'+str(i+1)] = np.where((trip17_vmt_F['STRTTIME'] >= 100*i) & (trip17_vmt_F['ENDTIME']< 100*(i+1)), 
                                               trip17_vmt_F['VMT_MILE']/(trip17_vmt_F['TRVLCMIN']/60),0)
                                               

In [ ]:
for i in range(24):
    trip17_vmt_T.loc[trip17_vmt_T['VMT_Hr'+str(i+1)] > 120,'VMT_Hr'+str(i+1)] = 120
    trip17_vmt_F.loc[trip17_vmt_F['VMT_Hr'+str(i+1)] > 120,'VMT_Hr'+str(i+1)] = 120

In [ ]:
trip17_vmt_T[['HOUSEID','VMT_Hr16','VMT_MILE']].head(10)

In [ ]:
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')
    

In [ ]:
trip17_vmt_T[trip17_vmt_T['VMT_Hr'+str(1+1)] != 0]['VMT_Hr2'].head()

In [ ]:
plt.figure(figsize=(16, 8))
plt.title('Time of Day VMT for personally driven vehicle trips', fontsize=28)
 
# data required for plotting
all_data = [trip17_vmt_T[trip17_vmt_T['VMT_Hr'+str(i+1)] != 0]['VMT_Hr'+str(i+1)] 
            for i in range(24)]
all_data_F = [trip17_vmt_F[trip17_vmt_F['VMT_Hr'+str(i+1)]!=0]['VMT_Hr'+str(i+1)] 
            for i in range(24)]

# plot box plot
flierprops1 = dict(marker='o', markerfacecolor='blue', markersize=6,
                  linestyle='none', markeredgecolor='g')
bp2 = plt.boxplot(all_data,positions=np.array(range(1,len(all_data)+1))*2-0.3, flierprops=flierprops1)
set_box_color(bp2, '#0000FF')


# plot box plot
flierprops2 = dict(marker='o', markerfacecolor='r', markersize=6,
                  linestyle='none', markeredgecolor='g')
bp4 = plt.boxplot(all_data_F,positions=np.array(range(1,len(all_data)+1))*2+0.3, flierprops=flierprops2)
#axs[1][1].set_title('Box plot')
set_box_color(bp4, '#FF0000')


plt.plot([], c='#0000FF', label="MUDs' Residents")
plt.plot([], c='#FF0000', label="Others' Residents")
plt.legend(loc='upper left', bbox_to_anchor=(0.9, 1.06))
# adding horizontal grid lines

plt.xticks(range(1, len(tod)*2+1, 2), tod)
plt.xticks(rotation=45)        
plt.xlabel('Time of Day (Hrs)',fontdict=font)
plt.ylabel('VMT (Miles)',fontdict=font)

plt.show();

In [ ]:
all_data[24].mean()

In [ ]:
list_avg_vmt_T = []
list_avg_vmt_F = []
for i in range(24):
    list_avg_vmt_T.append(all_data[i].mean())
    list_avg_vmt_F.append(all_data_F[i].mean())

In [ ]:
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')

plt.figure(figsize =(20,12))

plt.plot(tod, list_avg_vmt_T, label='MUDs');
plt.plot(tod, list_avg_vmt_F, label='Others');
#plt.plot(tod, avg_vmt[0], label='Alls');

plt.xlabel('Time of day (Hrs)', fontdict=font)
plt.ylabel('Average VMT (Miles)', fontdict=font)
plt.title('Time of Day Average VMTs for personally driven vehicle trips', fontsize=28)
plt.legend(loc='upper right')
plt.xticks(rotation=45)

plt.show()

In [ ]:
def avg_vmt_plot(avg_vmt_T,avg_vmt_F,division,tod):
    plt.figure(figsize =(20,12))

    plt.plot(tod, avg_vmt_T[0], label='MUDs');
    plt.plot(tod, avg_vmt_F[0], label='Others');
    #plt.plot(tod, avg_vmt[0], label='Alls');

    plt.xlabel('Time of day', fontdict=font)
    plt.ylabel('Average VMT (Miles)', fontdict=font)
    plt.title('Average VMTs of predicted MUDs of '+division, fontsize=28)
    plt.legend(loc='upper right')
    plt.xticks(rotation=45)

    plt.show()

In [ ]:
# plot 9 divisions in one graph of MUDs and Others respectively
def avg_vmt_plot_muds(avg_vmt_T,avg_vmt_F,division,tod,style_line,marker_fill):
    
    axs[0].plot(tod, avg_vmt_T[0], label=division,linestyle=style_line,marker=marker_fill)
    axs[1].plot(tod, avg_vmt_F[0], label=division,linestyle=style_line,marker=marker_fill)
    #plt.plot(tod, avg_vmt_F[0], label='Others');
    #plt.plot(tod, avg_vmt[0], label='Alls');

    
    axs[0].set_ylabel('Average VMT (Miles)', fontdict=font)
    axs[1].set_xlabel('Time of day (Hrs)', fontdict=font)
    axs[1].set_ylabel('Average VMT (Miles)', fontdict=font)
    axs[0].set_title('Average VMTs of predicted MUDs residents vehicles for different Census divisions', fontsize=28)
    axs[1].set_title('Average VMTs of other residents vehicles for different Census divisions', fontsize=28)
    axs[0].legend(loc='upper right')
    axs[1].legend(loc='upper right')
 
    axs[0].set_xticklabels(tod, rotation=45)
    axs[1].set_xticklabels(tod, rotation=45)

    #plt.show()

In [ ]:
avg_vmt_T = [[0 for i in range(24)] for j in range(100)]
trip17_vmt_CENSUS = trip17_vmt[trip17_vmt['CENSUS_D'] == 0+1]
trip17_vmt_T = trip17_vmt_CENSUS[trip17_vmt_CENSUS['MUDs_Predict_OW'] == 1]
for j in range(24):
    avg_vmt_T[0][j] = trip17_vmt_T[(trip17_vmt_T['STRTTIME'] >= 100*j) & (trip17_vmt_T['STRTTIME']< 100*(j+1))].VMT_MILE.mean()



In [ ]:
avg_vmt_T[0] = [0 if str(x)=='nan' else x for x in avg_vmt_T[0]]

In [ ]:
avg_vmt_T

In [ ]:
division_list = ['New England','Middle Atlantic','East North Central','West North Central',
                 'South Atlantic','East South Central','West South Central','Mountain','Pacific']

filled_markers = ['o', 'v', '8', 's', 'p', '*', 'H', 'D', 'X']
linestyle_tuple = ['solid','dashed','dashdot','dotted','solid',
                   'solid','dashed','dashdot','dotted']
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')


fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(20, 17))
for i in range(9):
    division = division_list[i]
    style_line = linestyle_tuple[i]
    marker_fill = filled_markers[i]
    avg_vmt_T = [[0 for i in range(24)] for j in range(100)]
    trip17_vmt_T_Census = trip17_vmt_T[trip17_vmt_T['CENSUS_D'] == i+1]
    #trip17_vmt_T = trip17_vmt_CENSUS[trip17_vmt_CENSUS['MUDs_Predict_OW'] == 1]
    for j in range(24):
#         avg_vmt_T[0][j] = trip17_vmt_T[(trip17_vmt_T['STRTTIME'] >= 100*j) & (trip17_vmt_T['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        '''
        Drop zeros VMT trips or not
        '''
        #temp_T = trip17_vmt_T_Census[trip17_vmt_T_Census['VMT_Hr'+str(j+1)] != 0]
        temp_T = trip17_vmt_T_Census
        avg_vmt_T[0][j] = temp_T['VMT_Hr'+str(j+1)].mean()
        
    avg_vmt_T[0] = [0 if str(x)=='nan' else x for x in avg_vmt_T[0]]
    
    avg_vmt_F = [[0 for i in range(24)] for j in range(100)]
    trip17_vmt_F_Census = trip17_vmt_F[trip17_vmt_F['CENSUS_D'] == i+1]
#     trip17_vmt_CENSUS = trip17_vmt[trip17_vmt['CENSUS_D'] == i+1]
#     trip17_vmt_F = trip17_vmt_CENSUS[trip17_vmt_CENSUS['MUDs_Predict_OW'] == 0]
    for j in range(24):
        #avg_vmt_F[0][j] = trip17_vmt_F[(trip17_vmt_F['STRTTIME'] >= 100*j) & (trip17_vmt_F['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        '''
        Drop zeros VMT trips or not
        '''
        #temp_F = trip17_vmt_F_Census[trip17_vmt_F_Census['VMT_Hr'+str(j+1)] != 0]
        temp_F = trip17_vmt_F_Census
        avg_vmt_F[0][j] = temp_F['VMT_Hr'+str(j+1)].mean()
    avg_vmt_F[0] = [0 if str(x)=='nan' else x for x in avg_vmt_F[0]]    
    
    avg_vmt_plot_muds(avg_vmt_T,avg_vmt_F,division,tod,style_line,marker_fill)
    

## Drop all zero VMTs trips

In [ ]:
division_list = ['New England','Middle Atlantic','East North Central','West North Central',
                 'South Atlantic','East South Central','West South Central','Mountain','Pacific']

filled_markers = ['o', 'v', '8', 's', 'p', '*', 'H', 'D', 'X']
linestyle_tuple = ['solid','dashed','dashdot','dotted','solid',
                   'solid','dashed','dashdot','dotted']
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')


fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(20, 17))
for i in range(9):
    division = division_list[i]
    style_line = linestyle_tuple[i]
    marker_fill = filled_markers[i]
    avg_vmt_T = [[0 for i in range(24)] for j in range(100)]
    trip17_vmt_T_Census = trip17_vmt_T[trip17_vmt_T['CENSUS_D'] == i+1]
    #trip17_vmt_T = trip17_vmt_CENSUS[trip17_vmt_CENSUS['MUDs_Predict_OW'] == 1]
    for j in range(24):
#         avg_vmt_T[0][j] = trip17_vmt_T[(trip17_vmt_T['STRTTIME'] >= 100*j) & (trip17_vmt_T['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        '''
        Drop zeros VMT trips or not
        '''
        temp_T = trip17_vmt_T_Census[trip17_vmt_T_Census['VMT_Hr'+str(j+1)] != 0]
        #temp_T = trip17_vmt_T_Census
        avg_vmt_T[0][j] = temp_T['VMT_Hr'+str(j+1)].mean()
        
    avg_vmt_T[0] = [0 if str(x)=='nan' else x for x in avg_vmt_T[0]]
    
    avg_vmt_F = [[0 for i in range(24)] for j in range(100)]
    trip17_vmt_F_Census = trip17_vmt_F[trip17_vmt_F['CENSUS_D'] == i+1]
#     trip17_vmt_CENSUS = trip17_vmt[trip17_vmt['CENSUS_D'] == i+1]
#     trip17_vmt_F = trip17_vmt_CENSUS[trip17_vmt_CENSUS['MUDs_Predict_OW'] == 0]
    for j in range(24):
        #avg_vmt_F[0][j] = trip17_vmt_F[(trip17_vmt_F['STRTTIME'] >= 100*j) & (trip17_vmt_F['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        '''
        Drop zeros VMT trips or not
        '''
        temp_F = trip17_vmt_F_Census[trip17_vmt_F_Census['VMT_Hr'+str(j+1)] != 0]
        #temp_F = trip17_vmt_F_Census
        avg_vmt_F[0][j] = temp_F['VMT_Hr'+str(j+1)].mean()
    avg_vmt_F[0] = [0 if str(x)=='nan' else x for x in avg_vmt_F[0]]    
    
    avg_vmt_plot_muds(avg_vmt_T,avg_vmt_F,division,tod,style_line,marker_fill)
    

In [ ]:
trip17_pred['NOHOMEDWELL']=np.where(trip17_pred['WHYTRP1S']!='01',trip17_pred['DWELTIME'],0)

In [ ]:
trip17_pred['NOHOMEDWELL']=np.where(2400-trip17_pred['ENDTIME']<trip17_pred['NOHOMEDWELL'],2400-trip17_pred['DWELTIME'],trip17_pred['NOHOMEDWELL'])
#trip17_MUD_final['NOHOMEDWELL']= min(2400-trip17_MUD_final['ENDTIME'],trip17_MUD_final['NOHOMEDWELL'])

In [ ]:
trip17_interme=trip17_pred.groupby(['HOUSEID'], as_index=False)[['TRVLCMIN','NOHOMEDWELL']].sum()

In [ ]:
trip17_interme['time_home'] = 1440 - trip17_interme['TRVLCMIN'] - trip17_interme['NOHOMEDWELL']

In [ ]:
trip17_interme['share_home'] = trip17_interme['time_home'] /1440

In [ ]:
trip17_pred_T = trip17_pred[trip17_pred['MUDs_Predict_OW'] == 1]
trip17_pred_F = trip17_pred[trip17_pred['MUDs_Predict_OW'] == 0]

In [ ]:
trip17_interme = trip17_interme[trip17_interme['share_home']>0]

In [ ]:
trip17_interme = trip17_interme[trip17_interme['share_home']<1]

In [ ]:
trip17_interme=trip17_interme.merge(trip17_pred[['HOUSEID','CENSUS_D']],on=['HOUSEID'],how='inner')



In [ ]:

trip17_interme = trip17_interme.groupby(['HOUSEID'], as_index=False).mean()

In [ ]:
trip17_interme['MUDs_OW'] = simu['MUDs_Predict_OW']

In [ ]:
trip17_interme.head()

In [ ]:
trip17_interme =trip17_interme[(trip17_interme['share_home']<=1) & (trip17_interme['share_home']>=0)]

In [ ]:
def box_share_home(my_dict,font,cate):
    fig, ax = plt.subplots(figsize = (16,8))
    ax.boxplot(my_dict.values())
    ax.set_xticklabels(my_dict.keys())
#    ax.set_ylim(0, 1);
    ax.set_xlabel('CENSUS DIVISIONS',fontdict= font)
    ax.set_ylabel('% of time of a day (0-1)',fontdict= font)
    ax.set_title('Share of time staying at home of ' + cate,fontdict= font)
    plt.show()
    

In [ ]:
division_list = ['New England','Middle Atlantic','East North Central','West North Central',
                 'South Atlantic','East South Central','West South Central','Mountain','Pacific']


data_dict = dict(zip(division_list, [None]*len(division_list)))

In [ ]:
cate = ['MUDs residents', 'Others residents']
trip17_MUD_1v_T = trip17_interme[trip17_interme['MUDs_OW'] == 1]
for i in range(9):
    data_dict[division_list[i]] = trip17_MUD_1v_T[trip17_MUD_1v_T['CENSUS_D'] == 1+i]['share_home']

box_share_home(data_dict,font,cate[0])

In [ ]:
trip17_MUD_1v_F = trip17_interme[trip17_interme['MUDs_OW'] == 0]
for i in range(9):
    data_dict[division_list[i]] = trip17_MUD_1v_F[trip17_MUD_1v_F['CENSUS_D'] == 1+i]['share_home']

box_share_home(data_dict,font,cate[1])

In [ ]:
division_list = ['New England','Middle Atlantic','East North Central','West North Central',
                 'South Atlantic','East South Central','West South Central','Mountain','Pacific']


data_dict = dict(zip(division_list, [None]*len(division_list)))
data_dict1 = dict(zip(division_list, [None]*len(division_list)))
data_dict2 = dict(zip(division_list, [None]*len(division_list)))

trip17_MUD_1v_T = trip17_interme[trip17_interme['MUDs_OW'] == 1]
for i in range(9):
    data_dict1[division_list[i]] = trip17_MUD_1v_T[trip17_MUD_1v_T['CENSUS_D'] == 1+i]['share_home']

trip17_MUD_1v_F = trip17_interme[trip17_interme['MUDs_OW'] == 0]
for i in range(9):
    data_dict2[division_list[i]] = trip17_MUD_1v_F[trip17_MUD_1v_F['CENSUS_D'] == 1+i]['share_home']



In [ ]:
data_dict1

In [ ]:
plt.figure(figsize=(16, 8))
plt.title('Share of time staying at home', fontsize=28)
 
# data required for plotting
all_data = [data_dict1[division_list[i]] for i in range(9)]
all_data_F = [data_dict2[division_list[i]] for i in range(9)]

# plot box plot
flierprops1 = dict(marker='o', markerfacecolor='blue', markersize=6,
                  linestyle='none', markeredgecolor='g')
bp2 = plt.boxplot(all_data,positions=np.array(range(1,len(all_data)+1))*2-0.3, flierprops=flierprops1)
set_box_color(bp2, '#0000FF')


# plot box plot
flierprops2 = dict(marker='o', markerfacecolor='r', markersize=6,
                  linestyle='none', markeredgecolor='g')
bp4 = plt.boxplot(all_data_F,positions=np.array(range(1,len(all_data)+1))*2+0.3, flierprops=flierprops2)
#axs[1][1].set_title('Box plot')
set_box_color(bp4, '#FF0000')


plt.plot([], c='#0000FF', label="MUDs' Residents")
plt.plot([], c='#FF0000', label="Others' Residents")
plt.legend(loc='upper left', bbox_to_anchor=(0.85, 1.06))
# adding horizontal grid lines

plt.xticks(range(2, len(division_list)*2 + 1, 2), division_list)
#plt.xticks(rotation=45)        
plt.xlabel('CENSUS DIVISIONS',fontdict=font)
plt.ylabel('% of time',fontdict=font)

plt.show()

In [ ]:
# MUDs: time not at home

In [ ]:
trip17_MUD_1v_vmt_T['VMT_MILE']

In [ ]:
trip17_MUD_1v_vmt_T = trip17_pred[trip17_pred['MUDs_Predict_OW'] == 1]
trip17_MUD_1v_vmt_F = trip17_pred[trip17_pred['MUDs_Predict_OW'] == 0]
trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt_T[
    np.abs(trip17_MUD_1v_vmt_T.VMT_MILE - trip17_MUD_1v_vmt_T.VMT_MILE.mean()) <= (3*trip17_MUD_1v_vmt_T.VMT_MILE.std())
]

trip17_MUD_1v_vmt_F = trip17_MUD_1v_vmt_F[
    np.abs(trip17_MUD_1v_vmt_F.VMT_MILE - trip17_MUD_1v_vmt_F.VMT_MILE.mean()) <= (3*trip17_MUD_1v_vmt_F.VMT_MILE.std())
]

plt.figure(figsize =(12,8))

sns.distplot(trip17_MUD_1v_vmt_T['VMT_MILE'],color= 'blue',label='VMT of MUDs');


sns.distplot(trip17_MUD_1v_vmt_F['VMT_MILE'],color="red",label = 'VMT of Others');
plt.xlabel('VMT (miles)',fontdict = font)
plt.ylabel("Density",fontdict= font)
plt.title('Distribution of Trip distance in miles for one primary vehicle per household', fontsize=24)
#.legend(loc='upper right');
plt.legend(loc='best');




In [ ]:

def vmt_count_dist(trip17_MUD_vmt_T, trip17_MUD_vmt_F,division):
    trip17_MUD_vmt_T = trip17_MUD_vmt_T[
        np.abs(trip17_MUD_vmt_T.VMT_MILE - trip17_MUD_vmt_T.VMT_MILE.mean()) <= (3*trip17_MUD_vmt_T.VMT_MILE.std())
    ]
    trip17_MUD_vmt_T = trip17_MUD_vmt_T.groupby('VMT_MILE').WTTRDFIN.sum().reset_index(name ='count')
    trip17_MUD_vmt_T = trip17_MUD_vmt_T.groupby(pd.cut(trip17_MUD_vmt_T.VMT_MILE, bins=np.arange(trip17_MUD_vmt_T.VMT_MILE.min(), trip17_MUD_vmt_T.VMT_MILE.max(), step=10),include_lowest=True)).sum()
    trip17_MUD_vmt_T = trip17_MUD_vmt_T.head(5)

    trip17_MUD_vmt_F = trip17_MUD_vmt_F[
        np.abs(trip17_MUD_vmt_F.VMT_MILE - trip17_MUD_vmt_F.VMT_MILE.mean()) <= (3*trip17_MUD_vmt_F.VMT_MILE.std())
    ]
    trip17_MUD_vmt_F = trip17_MUD_vmt_F.groupby('VMT_MILE').WTTRDFIN.sum().reset_index(name ='count')
    trip17_MUD_vmt_F = trip17_MUD_vmt_F.groupby(pd.cut(trip17_MUD_vmt_F.VMT_MILE, bins=np.arange(trip17_MUD_vmt_F.VMT_MILE.min(), trip17_MUD_vmt_F.VMT_MILE.max(), step=10),include_lowest=True)).sum()
    trip17_MUD_vmt_F=trip17_MUD_vmt_F.head(5)

    VMT_MILE = []
    for i in range(len(trip17_MUD_vmt_T)):
        VMT_MILE.append(str(10*i) + ' to ' + str(10*(i+1)))

    x = np.arange(len(VMT_MILE))  # the label locations
    width = 0.35       # the width of the bars: can also be len(x) sequence

    fig, ax = plt.subplots(figsize = (16,8))

    # rects1 = ax.bar(x - width/2, men_means, width, label='Men')
    # rects2 = ax.bar(x - width/2, women_means, width, label='Women')

    # plt.bar(VMT_MILE, trip17_MUD_vmt_T['count'],x - width/2, width, label='MUDs');
    # plt.bar(VMT_MILE, trip17_MUD_vmt_F['count'],x + width/2, width, label='Others');

    ax.bar(x - width/2, trip17_MUD_vmt_T['count'], width, label='MUDs');
    ax.bar(x + width/2, trip17_MUD_vmt_F['count'], width, label='Others');

    ax.set_xlabel('VMT_MILE(miles)', fontdict=font)
    ax.set_ylabel('Counts of trips', fontdict=font)
    ax.set_title('Distribution of counts of Trip distance in miles for personally driven vehicle trips of '+division, fontdict=font)
    ax.legend(loc='upper right')
    #ax.xticks(rotation=45)
    ax.set_xticks(x)
    ax.set_xticklabels(VMT_MILE)

    plt.show()

    
division_list = ['New England','Middle Atlantic','East North Central','West North Central',
                 'South Atlantic','East South Central','West South Central','Mountain','Pacific']

for i in range(9):
    trip17_MUD_vmt_CENSUS = trip17_pred[trip17_pred['CENSUS_D'] == i+1]
    trip17_MUD_vmt_T = trip17_MUD_vmt_CENSUS[trip17_MUD_vmt_CENSUS['MUDs_Predict_OW'] == 1]
    trip17_MUD_vmt_F = trip17_MUD_vmt_CENSUS[trip17_MUD_vmt_CENSUS['MUDs_Predict_OW'] == 0]

    division = division_list[i]
    vmt_count_dist(trip17_MUD_vmt_T, trip17_MUD_vmt_F,division)

In [ ]:
time_not_home = []
for i in range(len(hsid)):
    trip17_temp = trip17_pred_T[trip17_pred_T['HOUSEID'] == hsid[i]]
    temp1 = trip17_temp['TRVLCMIN'].sum()
    temp2 = min(trip17_temp[trip17_temp['WHYTRP1S'] != '01']['DWELTIME'],
                2400-trip17_temp[trip17_temp['WHYTRP1S'] != '01']['STRTTIME'])
    temp = temp1 + temp2
    time_not_home.append(temp)

In [ ]:
time_not_home = [i for i in time_not_home if i != 0]
time_home_muds = []
for i in range(len(time_not_home)):
    temp = (1440-time_not_home[i])/1440
    time_home_muds.append(temp)


In [ ]:
# Others: time not at home

In [ ]:
time_not_home_others = []
for i in range(len(hsid)):
    temp1 = trip17_pred_F[trip17_pred_F['HOUSEID'] == hsid[i]]['TRVLCMIN'].sum()
    temp2 = trip17_pred_F[(trip17_pred_F['HOUSEID'] == hsid[i]) &
                          (trip17_pred_F['WHYTRP1S'] != '01')]['DWELTIME'].sum()
    temp = temp1 + temp2
    time_not_home_others.append(temp)

In [ ]:
time_not_home_others = [i for i in time_not_home_others if i != 0]
time_home_others = []
for i in range(len(time_not_home_others)):
    temp = (1440-time_not_home_others[i])/1440
    time_home_others.append(temp)